<a href="https://colab.research.google.com/github/JinFree/darknet/blob/master/Train_DarkNet_Yolov4_tiny_google_openimages_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

별도로 복사해 둔 DarkNet 저장소를 받아온 후 빌드하겠습니다.
nvidia-smi를 통해 나오는 GPU Name을 확인해두어야 합니다.

In [ ]:
!git clone https://github.com/JinFree/darknet.git
!cp darknet/Makefile_Colab darknet/Makefile 
!nvidia-smi

DarkNet을 빌드합니다.
만약 빌드 중 문제가 생기면 Makefile의 20번 줄 ARCH을 확인한 후 위에서 확인한 GPU 이름의 CUDA Compute Capability를 확인하여 수정해주시면 되겠습니다.

In [ ]:
!cd darknet && make -j$(nproc)

아래 셀을 실행했을 때 bycycle, person, dog, truck, car에 대한 객체 인식 결과를 확인할 수 있다면 DarkNet 빌드가 정상적으로 수행 된 것입니다.

아래와 유사하게 출력됩니다.

----

data/dog.jpg: Predicted in 5.928000 milli-seconds.

bicycle: 29%

person: 25%

dog: 72%

truck: 82%

car: 46%

----


In [ ]:
!cd darknet \
&& ./darknet detect cfg/yolov4-tiny.cfg bin/yolov4-tiny.weights data/dog.jpg

데이터셋을 준비하겠습니다. 약 5분이 소요됩니다.

In [ ]:
!git clone https://github.com/JinFree/Openimages2YOLO
!python3 -m pip install --upgrade pip
!python3 -m pip install boto3 botocore 

Openimages2YOLO 저장소 내의 open_images_downloader.py를 실행합니다. 이 때  --class-names와  --max-images를 상황에 맞게 수정합니다.

In [ ]:
!python3 Openimages2YOLO/open_images_downloader.py  --class-names "Person, Car" --data=/content/data  --max-images=5000 --num-workers=2

Openimages2YOLO 저장소 내의 convert_to_yolo.py를 실행합니다. 
실행하기 전에 해당 스크립트 내의 dataset_root와 classes를 사용 환경에 맞게 조절합니다.

In [ ]:
!python3 Openimages2YOLO/convert_to_yolo.py

Yolov4-tiny 훈련을 위해 전이학습할 가중치를 받아온 후 다운받은 데이터셋을 학습할 수 있도록 수정한 yolov4-tiny_custom.cfg 파일을 준비한 후 수정합니다

아래는 yolov4-tiny_custom.cfg 파일에 대한 설명입니다.

----

해당 파일 내 있는 두 개의 yolo 레이어에서 클래스 수를 2로 수정했습니다.

이후 yolo레이어 바로 위의 convolutional 레이어에서 filters 수를 21로 수정했습니다.

21는 ( 5 + 2 ) * 3을 통해 계산되었습니다.

----

In [ ]:
!mkdir backup && cd backup && wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!cp darknet/cfg/yolov4-tiny.cfg ./yolov4-tiny_custom.cfg

훈련을 위한 data_colab.txt 파일을 준비합니다.

훈련할 클래스의 수, 훈련할 데이터셋의 리스트가 적힌 파일의 경로, 클래스의 이름, 훈련 중 가중치가 저장될 폴더의 경로가 아래와 같이 적혀있습니다.

---

classes = 2

train = /content/data/train.txt

valid = /content/data/validation.txt

names = /content/data/class.txt

backup = /content/backup

---

훈련을 수행합니다.

훈련은 약 두시간정도 수행하면 됩니다.

훈련하면서 출력되는 로그는 train_log.txt로 저장됩니다.

In [ ]:
!cd darknet && ./darknet detector train ../data_colab.txt ../yolov4-tiny_custom.cfg ../backup/yolov4-tiny.conv.29 -dont_show  2&> train_log.txt

훈련이 어느정도 수행되면 훈련을 중지하셔도 됩니다.

직접 추론하기 위해서 필요한 파일은 아래의 네 가지 파일입니다.

----

darknet/vocdata.txt

vocnames.txt

yolov4-tiny_voc.cfg

backup/yolov4-tiny_voc_last.weights

----

만약 추가적으로 학습을 하고자 하는 경우, 아래의 셀을 실행하면 됩니다.

In [ ]:
!cd darknet && ./darknet detector train ../data_colab.txt ../yolov4-tiny_custom.cfg ../backup/yolov4-tiny_custom_last.weights -dont_show  2&> train_log.txt